In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.keras import Sequential
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
%load_ext tensorboard

In [3]:
!del .\logs

Could Not Find c:\Users\Jason\PycharmProjects\NoteClassification\logs


In [4]:
logdir = "logs\\hparam_tuning\\"

In [5]:
data = np.load("data.npy")
labels = np.load("labels.npy")

In [6]:
train_dataset_size = int(data.shape[0] * 0.85)
test_dataset_size = data.shape[0] - train_dataset_size

In [7]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.shuffle(data.shape[0])

train_dataset = dataset.take(train_dataset_size)
test_dataset = dataset.skip(train_dataset_size)

In [8]:
print(dataset)

<ShuffleDataset shapes: ((1025, 173, 3), ()), types: (tf.complex128, tf.string)>


In [23]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
HP_CONV_KERNEL_SIZE = hp.HParam("conv_kernel_size", hp.Discrete([3, 5]))
HP_POOL_STRIDE_SIZE = hp.HParam("pool_stride_size", hp.Discrete([2, 6]))
HP_POOL_WINDOW_SIZE = hp.HParam("pool_window_size", hp.Discrete([2, 6]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.3))
HP_DENSE_SIZE = hp.HParam("dense_size", hp.Discrete([100, 500]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
HP_EPOCH_SIZE = hp.HParam("epoch_size", hp.Discrete([50, 400]))
HP_BATCH_SIZE = hp.HParam("batch_sizes", hp.Discrete([16, 64]))

METRIC_ACCURACY = 'accuracy'
CATEGORICAL_ACCURACY = "categorical_crossentropy_accuracy"

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_CONV_KERNEL_SIZE, HP_POOL_STRIDE_SIZE, HP_POOL_WINDOW_SIZE, HP_DROPOUT, HP_DENSE_SIZE, HP_OPTIMIZER, HP_EPOCH_SIZE, HP_BATCH_SIZE],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy'), hp.Metric(CATEGORICAL_ACCURACY, display_name='Cross-Entropy Accuracy')],
  )

In [27]:
def train_test_model(hparams):
  model = Sequential(
    [
        Conv2D(hparams[HP_NUM_UNITS], int(hparams[HP_CONV_KERNEL_SIZE]), padding="same", activation="relu", input_shape=(1025, 173, 3)),
        MaxPooling2D(int(hparams[HP_POOL_WINDOW_SIZE]), strides= int(hparams[HP_POOL_STRIDE_SIZE])),
        Conv2D(32, int(hparams[HP_CONV_KERNEL_SIZE]), padding="same", activation="relu"),
        MaxPooling2D(int(hparams[HP_POOL_WINDOW_SIZE]), strides= int(hparams[HP_POOL_STRIDE_SIZE])),
        Conv2D(64, int(hparams[HP_CONV_KERNEL_SIZE]), padding="same", activation="relu"),
        MaxPooling2D(int(hparams[HP_POOL_WINDOW_SIZE]), strides= int(hparams[HP_POOL_STRIDE_SIZE])),
        Flatten(),
        Dense(hparams[HP_DENSE_SIZE], activation="relu"),
        Dropout(hparams[HP_DROPOUT]),
        Dense(65, activation="softmax")
    ]
  )
  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='categorical_crossentropy',
      metrics=['accuracy', 'categorical_crossentropy_accuracy'],
  )

  model.fit(train_dataset,
    batch_size=int(hparams[HP_BATCH_SIZE]),
    epochs=int(hparams[HP_EPOCH_SIZE]),
    callbacks=[
        tf.keras.callbacks.TensorBoard(logdir),  # log metrics
        hp.KerasCallback(logdir, hparams),  # log hparams
    ]
  )
  _, accuracy = model.evaluate(test_dataset)
  return accuracy

In [ ]:
session_num = 0
models = dict()

for num_units in HP_NUM_UNITS.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
        for epoch_size in np.linspace(HP_EPOCH_SIZE.domain.values[0], HP_EPOCH_SIZE.domain.values[1], 3):
            for batch_size in np.linspace(HP_BATCH_SIZE.domain.values[0], HP_BATCH_SIZE.domain.values[1], 4):
                for kernel_size in np.linspace(HP_CONV_KERNEL_SIZE.domain.values[0], HP_CONV_KERNEL_SIZE.domain.values[1], 3):
                    for stride_size in np.linspace(HP_POOL_STRIDE_SIZE.domain.values[0], HP_POOL_STRIDE_SIZE.domain.values[1], 5):
                        for window_size in np.linspace(HP_POOL_WINDOW_SIZE.domain.values[0], HP_POOL_WINDOW_SIZE.domain.values[1], 5):
                            for dropout_rate in np.linspace(HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value, 2):
                                for dense_size in np.linspace(HP_DENSE_SIZE.domain.values[0], HP_DENSE_SIZE.domain.values[1], 5):
                                    hparams = {
                                        HP_NUM_UNITS: num_units,
                                        HP_DROPOUT: dropout_rate,
                                        HP_OPTIMIZER: optimizer,
                                        HP_EPOCH_SIZE: epoch_size,
                                        HP_BATCH_SIZE: batch_size,
                                        HP_CONV_KERNEL_SIZE: kernel_size,
                                        HP_POOL_STRIDE_SIZE: stride_size,
                                        HP_POOL_WINDOW_SIZE: window_size,
                                        HP_DENSE_SIZE: dense_size,
                                    }
                                    run_name = "run-%d" % session_num
                                    print('--- Starting trial: %s' % run_name)
                                    print({h.name: hparams[h] for h in hparams})
                                    train_test_model(hparams)
                                    session_num += 1

[]